<a href="https://colab.research.google.com/github/Yahia-allal/First-Project/blob/main/credit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# importer les packages
import pandas as pd   #  Pandas est une bibliothèque, utilisée pour le traitement et l'analyse des données
import numpy as np    # NumPy est une bibliothèque utiliser  pour la manipulation de données sous forme de tableaux multidimensionnels.
import matplotlib.pyplot as plt  # Matplotlib.pyplot est un module de la bibliothèque Matplotlib en Python, qui permet de créer des graphiques et des visualisations de données de manière concise et flexible

### **Je vais procéder au nettoyage de la base de données**

In [4]:
# lire la base de données
df=pd.read_csv('/content/train_loan-predication.csv')
pd.set_option('display.max_row',10) # pour afficher 10 lignes
df
# Identifiant d'un crédit, Genre, Marié(e), le nombre d'enfants, est ce que cette pers fait les études ou pas , est qu'ilest salarié  , Revenu, Revenu du conjoint,
#LoanAmount:Montant demander, Durée du prêt, est ce que cette perso a déja un crédit

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [5]:
df.info()
# Pour Voir les valeurs manquantes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
df.isnull().sum().sort_values(ascending=False)  # ascending=False:les valeurs seront triées dans l'ordre décroissant
# Chercher dans ma base de données les valeurs manquantes (null), les additionner et les classer(combien de valeur manquantes pour chaque attribut)

Credit_History       50
Self_Employed        32
LoanAmount           22
Dependents           15
Loan_Amount_Term     14
                     ..
Education             0
ApplicantIncome       0
CoapplicantIncome     0
Property_Area         0
Loan_Status           0
Length: 13, dtype: int64

In [7]:
df.describe()
# statistique a ma base de donnée

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [8]:
df.describe(include='object')
#affichera les statistiques des colonnes de type objet(catégorique) dans la base de donnée

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
count,614,601,611,599,614,582,614,614
unique,614,2,2,4,2,2,3,2
top,LP001002,Male,Yes,0,Graduate,No,Semiurban,Y
freq,1,489,398,345,480,500,233,422


In [9]:
# Puisque On a un nombre important de valeurs manquantes donc on ne peut pas supprimer. Dans ce cas
# je vais renseigner les valeurs manquantes.

# Renseingner les valeurs manquantes
cat_data=[]    #Créer une liste pour une data categorique
num_data=[]    #Créer une liste pour une data numérique

for i,c in enumerate(df.dtypes):             # 'df.dtypes' retourne les types de données de chaque colonne
  if c==object:
    cat_data.append(df.iloc[:,i])       # iloc pour choisir une colonne d'ordre i
  else:
      num_data.append(df.iloc[:,i])

cat_data = pd.DataFrame(cat_data).transpose()    # la methode 'DataFrame' Transformer une liste a une base de données
num_data=pd.DataFrame(num_data).transpose()
df.dtypes

Loan_ID              object
Gender               object
Married              object
Dependents           object
Education            object
                     ...   
LoanAmount          float64
Loan_Amount_Term    float64
Credit_History      float64
Property_Area        object
Loan_Status          object
Length: 13, dtype: object

In [10]:
num_data

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,5849.0,0.0,NaN,360.0,1.0
1,4583.0,1508.0,128.0,360.0,1.0
2,3000.0,0.0,66.0,360.0,1.0
3,2583.0,2358.0,120.0,360.0,1.0
4,6000.0,0.0,141.0,360.0,1.0
...,...,...,...,...,...
609,2900.0,0.0,71.0,360.0,1.0
610,4106.0,0.0,40.0,180.0,1.0
611,8072.0,240.0,253.0,360.0,1.0
612,7583.0,0.0,187.0,360.0,1.0


In [11]:
cat_data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,Urban,Y
4,LP001008,Male,No,0,Graduate,No,Urban,Y
...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,Urban,Y


In [12]:
# Pour les variables catégoriques on va remplacer les valeurs manquantes par les valeurs qui se repetent le plus
cat_data = cat_data.apply(lambda x:x.fillna(x.value_counts().index[0]))


cat_data.isnull().sum().any()   # pour vérifier est ce qu'il ya les valeurs manquantes ou non

False

In [13]:
 # Pour les variables numériques on va remplacer les valeurs manquantes par la valeur précedente de la meme colonne

num_data.fillna(method='bfill',inplace=True)
num_data.isnull().sum().any()

False

In [14]:
# Transformer la colonne target Y par 1 et N par 0

target_value = {'Y':1, 'N':0}
target = cat_data['Loan_Status']
cat_data.drop('Loan_Status',axis=1,inplace=True)
target=target.map(target_value)  #La méthode map en pandas est utilisée pour transformer les valeurs d'une colonne

target

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

In [15]:
# Donc pour Remplacer les valeurs catégoriques par des valeurs numérique 0, 1, 2,....
# je vais importer la methode LabelEncoder de Sklearn, qui utilisée pour convertir des étiquettes de classes textuelles en nombres entiers

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


for i in cat_data:     # Cette boucle pour convertir les valeurs catégoriques
  cat_data[i] =le.fit_transform(cat_data[i])
cat_data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,0,1,0,0,0,0,2
1,1,1,1,1,0,0,0
2,2,1,1,0,0,1,2
3,3,1,1,0,1,0,2
4,4,1,0,0,0,0,2
...,...,...,...,...,...,...,...
609,609,0,0,0,0,0,0
610,610,1,1,3,0,0,0
611,611,1,1,1,0,0,2
612,612,1,1,2,0,0,2


In [16]:
#Supprimer  (Loan_id) car répétée deux fois
cat_data.drop('Loan_ID',axis=1,inplace=True)

In [17]:
# cette ligne Pour Concatener cat_data et num_data et spécifier la colonne target

x=pd.concat([cat_data,num_data],axis=1)
y=target

In [18]:
x

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,1,0,0,0,0,2,5849.0,0.0,128.0,360.0,1.0
1,1,1,1,0,0,0,4583.0,1508.0,128.0,360.0,1.0
2,1,1,0,0,1,2,3000.0,0.0,66.0,360.0,1.0
3,1,1,0,1,0,2,2583.0,2358.0,120.0,360.0,1.0
4,1,0,0,0,0,2,6000.0,0.0,141.0,360.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,0,2900.0,0.0,71.0,360.0,1.0
610,1,1,3,0,0,0,4106.0,0.0,40.0,180.0,1.0
611,1,1,1,0,0,2,8072.0,240.0,253.0,360.0,1.0
612,1,1,2,0,0,2,7583.0,0.0,187.0,360.0,1.0


In [19]:
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64


### je vais essayer de faire une analyse exploratoire (c'est a dire comprendre notre base de données)


In [ ]:
## On va commencer par la variable target

target.value_counts()    # Combien de Crédit accépter et refuser



In [ ]:
# voila La base de données qui utilisée pour l'analyse d'exploratoire

df=pd.concat([cat_data,num_data,target],axis=1)
df

In [ ]:
### La visualisation de la data ##

 #je vais importer Seaborn, est une bibliothèque utilisée pour la visualisation de données,et créer des graphiques statistiques
import seaborn as sns

# donc je vais Tracer le plot de Distribution des catégories dans la variable target (Loan Status)

target_df = pd.DataFrame({'target': target})
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=target_df,palette='Set1')

# Ajouter des étiquettes et un titre
plt.xlabel('Loan_Status')
plt.ylabel('count')
plt.title('Distribution des catégories dans la variable Loan Status')
plt.show()
#le pourcetage des crédit accordés
yes=target.value_counts()[0]/len(target)
#le pourcetage des crédit non accordés
no=target.value_counts()[1]/len(target)
print(f'le pourcetage des crédit accordés est: {yes}')
print(f'le pourcetage des crédit non accordés est: {no}')

In [ ]:
#Je peux aussi parcourir variable par variable pour voir la relation entre elles.

#Par exemple, dans ce graphe qui va indiquer la relation entre le fait d'accepter un crédit ou non et le fait d'avoir déjà un crédit.
grid = sns.FacetGrid(df, col='Loan_Status', height=3.2, aspect=1.6)
grid.map(sns.countplot, 'Credit_History')

plt.show()


In [ ]:
#Donc remarquez que l'on accepte le crédit pour qu'ont déjà un historique de crédit.

In [ ]:
# On peut aussi voir si (Education) l'éducation a un impact ou pas.

grid=sns.FacetGrid(df, col='Loan_Status', height=3.2, aspect=1.6)
grid.map(sns.countplot, 'Education')
plt.show()


In [ ]:
# voir que les gens qu'ont fait des etudes ont un crédit accépter mais les autres non (Avec 0 pour qu'est on fait les études)
# Donc c'est une attribut également qu'on peut intégrie

In [ ]:
# Aussi entre le Sexe est target
grid = sns.FacetGrid(df, col='Loan_Status', height=3.2, aspect=1.6)
grid.map(sns.countplot, 'Gender')
plt.show()

In [ ]:
   # Remerquer que généralement les hommes qu'est demande plus de crédit et qu'ont plus de crédit accépter

In [ ]:
# Dans un variable numérique par exemple le revenue de demendeur (applicantIncome)

plt.scatter(df['ApplicantIncome'],df['Loan_Status'])


In [ ]:
#Remarque ici n'a pas un impact car généralement les revenues se retrouve de meme coté
#c'est a dire on ne trouve pas une concentration dans un type de revenue

In [ ]:
### on peut faire autre choses

#un groupe by sur notre target (Loan_Status) et faire la moyenne
df.groupby('Loan_Status').median()


In [ ]:
# Et Remarque que un impacte de le revenue de conjoint (CoapplicatIcome), car on a une différence entre la moyenne lorsque le Crédit accépter ou pas

### Aprés l'analyse d'exploratoire sur la base de données, Manitenant on va realiser un modèle on se basent sur les algorithmes de machine learning

In [ ]:
# Premièrement On va déviser la base de données en une base de données d'entrainement et de test


In [ ]:
# Une Remarque : la methode classique de sklearn et 'train_test_split'  généralement utilisé pour une division simple et aléatoire,
#mais la methode StratifiedShuffleSplit utilisé pour une division où la préservation de la distribution des classes est essentielle
#surtout lorsque les classes sont déséquilibrées.


from sklearn.model_selection import StratifiedShuffleSplit  # je vais importer la methode StratifiedShuffleSplit pour diviser la data

sss=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42) # 20% pour la partie test


for train,test in sss.split(x,y):       # je vais Créer une boucle qui va faire cette division
  x_train,x_test=x.iloc[train],x.iloc[test]
  y_train,y_test=y.iloc[train],y.iloc[test]

print('x_train taille: ', x_train.shape)
print('x_test taille: ', x_test.shape)
print('y_train taille: ', y_train.shape)
print('y_test taille: ', y_test.shape)

On va appliquer trois algorithmes Logistic Regression, KNN, DecsionTree pour cette classification et on va les evaluer  

In [ ]:
# alors je vais faire l'importation des modéles
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# j'ia Créer un dectionnaire et déclarer les modèles
models={
    'LogisticRegression':LogisticRegression(random_state=42),
    'KNeighborsClassifier':KNeighborsClassifier(),
    'DecisionTreeClassifier':DecisionTreeClassifier(max_depth=1,random_state=42),
}


from sklearn.metrics import accuracy_score   # je vais impoter une métrique


def accu(y_true, y_pred):              # Et j'ai définie une fonction de précesion

    acc = accuracy_score(y_true, y_pred)
    print(f'la précision du modèle est : {acc}')

def train_test_eval(models,x_train,y_train,x_test,y_test):    # et on va définie une focntion d'application des modèles

  for name,model in models.items():
    print(name,':')
    model.fit(x_train,y_train)
    accu(y_test,model.predict(x_test))
    print('-'*30)

# Faire l'appele de focntion d'application des modèles
train_test_eval(models,x_train,y_train,x_test,y_test)

In [ ]:
NewData=x[['Credit_History','Married','CoapplicantIncome']]

In [ ]:
sss=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42) # 20% pour la partie test

for train,test in sss.split(NewData,y):
  NewData_train,NewData_test=NewData.iloc[train],NewData.iloc[test]
  y_train,y_test=y.iloc[train],y.iloc[test]

print('NewData_train taille: ',NewData_train.shape)
print('NewData_test taille: ',NewData_test.shape)
print('y_train taille: ', y_train.shape)
print('y_test taille: ', y_test.shape)


### ***Le meilleur modèle est LogisticRegression, donc je vais la reprendre et l'appliquer seule au lieu d'appliquer les trois***




Finalement, je vais choisir quelques facteurs à utiliser pour ce modèle dans notre application

In [ ]:
#Appliquer LogisticRegression sur notre base de donnee
Classifier=LogisticRegression()
Classifier.fit(NewData,y)

In [ ]:
#Enregistrer le modele
#Pickle c'est un package utiliser pour enregistrer le modele
import pickle

pickle.dump(Classifier,open('model.pkl','wb'))

